In [2]:
import numpy as np
from my_ueg import my_ueg
from pyscf import gto, scf, cc

In [13]:
nocc = 19 # 7, 19, 27, 33, 57, 81, 93
my_sys = my_ueg(rs=4.0, nelec=(nocc, nocc), gamma=np.sqrt(2)) #np.sqrt(2))
print('num kpts: ',my_sys.nkpts)
h0 = my_sys.madelung() / 2
h1 = my_sys.get_h1_real()
cderi = my_sys.get_cderi_real()

mol = gto.M()
mol.nelectron = my_sys.nparticle
mol.nao = my_sys.nkpts
mol.incore_anyway = True
mol.max_memory = 20000
mol.verbose = 4

dm = np.zeros((my_sys.nkpts,my_sys.nkpts))
dm[:nocc,:nocc] = np.eye(nocc) * 2.0

mf = scf.RHF(mol).density_fit()
mf.energy_nuc = lambda *args: h0
mf.get_hcore = lambda *args: h1
mf.get_ovlp = lambda *args: np.eye(my_sys.nkpts)
mf._cderi = cderi
mf.init_guess = "1e"
mf.max_cycle = -1
escf = mf.kernel(dm0=dm)

mf.mo_coeff = np.eye(my_sys.nkpts)

print(f"escf: {escf}")
print(f"escf per electron: {escf/my_sys.nparticle}")

mycc = cc.RCCSD(mf)
mycc.max_cycle = 100
mycc.conv_tol = 1e-7
mycc.kernel()

# et = mycc.ccsd_t()
print("CCSD energy", mycc.e_corr)
print("CCSD energy per electron", mycc.e_corr / my_sys.nparticle)
# print("CCSD(T) energy", et)
# print("CCSD(T) energy per electron", et / my_sys.nparticle)

num kpts:  57


******** <class 'pyscf.df.df_jk.DFRHF'> ********
method = DFRHF
initial guess = 1e
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = -1
direct_scf = False
chkfile to save SCF result = /tmp/tmplp2i0yt_
max_memory 20000 MB (current use 589 MB)
Set gradient conv threshold to 3.16228e-05
init E= -2.1778869872131
  HOMO = -0.0166962344253906  LUMO = 0.0251858185256435
SCF not converged.
SCF energy = -2.1778869872131
escf: -2.177886987213102
escf per electron: -0.05731281545297637

******** <class 'pyscf.cc.dfccsd.RCCSD'> ********
CC2 = 0
CCSD nocc = 19, nmo = 57
max_cycle = 100
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 20000 MB (current use 589 MB)
Init t2, MP2 energy = -3.35736141351301  E_corr(MP2) -1.17947442629991
Init E_corr(RCCSD) = -1.1794744

In [6]:
from ueg import ueg_qc as ak_ueg
from pyscf import ao2mo

In [12]:
nocc = 19 # 7, 19, 27, 33, 57, 81, 93
my_sys = my_ueg(rs=4.0, nelec=(nocc, nocc), gamma=np.sqrt(2))
system = ak_ueg(4.0, (nocc, nocc), e_cut_red=5) # this e_cut is for ueg_qc().get_k_points()
k_points = my_sys.get_kpts() # get k_pts by my_ueg, this allows k_cut = gamma*k_fermi
n_kpts = k_points.shape[0]
print(f"number of k-points: {n_kpts}")
h0 = system.madelung() / 2
h1 = system.get_h1_real(k_points)
eri = system.get_eri_tensor_real(k_points)
eri = np.array(eri)

mol = gto.M()
mol.nelectron = system.n_particles
mol.nao = n_kpts
mol.incore_anyway = True
mol.max_memory = 20000
mol.verbose = 4

# dm = np.zeros((n_kpts,n_kpts))
# dm[:nocc,:nocc] = np.eye(nocc) * 2.0

mf = scf.RHF(mol)
mf.energy_nuc = lambda *args: h0
mf.get_hcore = lambda *args: h1
mf.get_ovlp = lambda *args: np.eye(n_kpts)
# mf.get_jk = get_jk
mf.verbose = 4
mf._eri = ao2mo.restore(8, eri, n_kpts)
mf.init_guess = "1e"
mf.max_cycle = -1
# mf.mo_coeff = np.eye(n_kpts)
escf = mf.kernel()

# mf.mo_coeff = np.eye(n_kpts)

print(f"escf: {escf}")
print(f"escf per electron: {escf/system.n_particles}")

mycc = cc.RCCSD(mf)
mycc.max_cycle = 100
# mycc1.conv_tol = 1e-7
mycc.kernel()
print("CCSD energy", mycc.e_tot)
print("CCSD E_corr per electron", mycc.e_corr / system.n_particles)
print("CCSD energy per electron", mycc.e_tot / system.n_particles)

number of k-points: 57


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = 1e
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = -1
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmp6rm_3yi3
max_memory 20000 MB (current use 524 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from hcore.
  HOMO = 0.0840088326750951  LUMO = 0.126013249012643
init E= -2.1778869872131
  HOMO = -0.0166962344253908  LUMO = 0.0251858185256434
SCF not converged.
SCF energy = -2.1778869872131
escf: -2.1778869872131037
escf per electron: -0.057312815452976416

******** <class 'pyscf.cc.ccsd.CCSD'> ********
CC2 = 0
CCSD nocc = 19, nmo = 57
max_cycle = 100
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 20000 MB (current use 524 MB